# Project_ML

## Business understanding

### Problem statement
Breast cancer is one of the most common spread and Dangerous cancers worldwide . and the process of manual diagnosis and waiting for the Professional to analyze, while its not 100% effective , can be time consuming and subject to human error . thats why there is an essential need to help the medical Equipment by providing accurate and reliable decision support that assist and help classify the tumors detected as benign or malignant.
To address this need,machine Learning has been applied to the Wisconsin Diagnostic Breast Cancer (WDBC) dataset for breast cancer classification. specifically , the recent case study by Department of Computer Science Adamson University Manila, Philippines where researchers used GRU-SVM, Linear Regression, Multilayer Perceptron (MLP) and others. by measuring and acheiving an acuracy test we can tell that they achieved excelent results in telling benign and malignant tumors apart . and as we know this technology isnt to work alone . its designed as smart assistant for professionals to make their work faster. the main users are : pathologists , oncologists , gynaecologist

### Storytelling bos:
Ilef find that she has a mass in her breast so she decided to check with an oncologist, he asked her to do an fna test ,after the result he presented to her te new ai system 'ena healthy ' analysed her test and find out if it's a benign or malignant tumor,ilef panicked, she was afraid of  having a malignant tumor but the doctor calmed her and informed her that it is only an early warning because the system could also tell him the stage of tumor ,and means that they have to extract the mass by a surgery and then they will analyse it. So she booked an appointment with a surgent oncologist. Then a pathologists analyses the mass and 'ena healthy' told them if there is a risk in the future or not. After that she goes to gynaecologist to do frequent check ups to know the risk in the future.  
 
#### BOS:
1) Identify the type of tumor (benign or malignant)
2) Identify the stage of tumor
3) Identify the risk in the future
### DSOS:
1) dso1 :analyze FNA sample features to instantly diagnose a tumor as Benign or Malignant.
2) Predict the cancer stage (I, II, III, or IV) by examining the morphology of biopsy cells
3) dso 3 : estimate the probability of a malignant breast cancer in the future .



GRU-SVM

In [7]:
import pandas as pd
from IPython.display import display, Markdown

# Titre
display(Markdown("### Tableau récapitulatif du modèle GRU-SVM (Agarap, 2018)"))

# Tableau propre avec séparation hyperparamètres / paramètres appris
data = {
    "Modèle": ["GRU-SVM"],
    "Variables impliquées\n(30 features du WDBC)": [
        "radius_mean, texture_mean, perimeter_mean, area_mean, smoothness_mean,\n"
        "compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean,\n"
        "radius_se, texture_se, perimeter_se, area_se, smoothness_se,\n"
        "compactness_se, concavity_se, concave points_se, symmetry_se, fractal_dimension_se,\n"
        "radius_worst, texture_worst, perimeter_worst, area_worst, smoothness_worst,\n"
        "compactness_worst, concavity_worst, concave points_worst, symmetry_worst, fractal_dimension_worst"
    ],
    "Hyperparamètres\n(choisis avant entraînement)": [
        "• Batch Size : 128\n"
        "• GRU Cell Size : 128\n"
        "• Dropout Rate : 0.5\n"
        "• Epochs : 3000\n"
        "• Learning Rate : 0.001\n"
        "• Optimizer : Adam\n"
        "• SVM C (pénalité) : 5\n"
        "• Norm SVM : L2"
    ],
    "Paramètres appris\n(durant l'entraînement)": [
        "• Poids des gates GRU :\n"
        "   W_z, U_z, W_r, U_r, W_h, U_h\n"
        "• Poids du classifieur SVM : w\n"
        "• Biais SVM : b\n"
        "→ Optimisés par Adam en minimisant\n"
        "   la perte L2-SVM (Eq. 20 de l'article)"
    ]
}

df = pd.DataFrame(data)

# Affichage propre et sans erreur
display(df.style
    .set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
        'font-size': '14px',
        'padding': '10px'
    })
    .set_table_styles([
        {'selector': 'th', 
         'props': 'background-color: #2E86C1; color: white; font-weight: bold; text-align: center;'},
        {'selector': 'td', 
         'props': 'border: 1px solid #ddd; padding: 12px;'},
        {'selector': 'table', 
         'props': 'width: 100%; border-collapse: collapse; margin: 20px 0;'}
    ])
    .set_caption("Source : Agarap, A.F.M. (2018) - On Breast Cancer Detection")
)

### Tableau récapitulatif du modèle GRU-SVM (Agarap, 2018)

,Modèle,Variables impliquées (30 features du WDBC),Hyperparamètres (choisis avant entraînement),Paramètres appris (durant l'entraînement)
0,GRU-SVM,"radius_mean, texture_mean, perimeter_mean, area_mean, smoothness_mean, compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean, radius_se, texture_se, perimeter_se, area_se, smoothness_se, compactness_se, concavity_se, concave points_se, symmetry_se, fractal_dimension_se, radius_worst, texture_worst, perimeter_worst, area_worst, smoothness_worst, compactness_worst, concavity_worst, concave points_worst, symmetry_worst, fractal_dimension_worst",• Batch Size : 128 • GRU Cell Size : 128 • Dropout Rate : 0.5 • Epochs : 3000 • Learning Rate : 0.001 • Optimizer : Adam • SVM C (pénalité) : 5 • Norm SVM : L2,"• Poids des gates GRU : W_z, U_z, W_r, U_r, W_h, U_h • Poids du classifieur SVM : w • Biais SVM : b → Optimisés par Adam en minimisant la perte L2-SVM (Eq. 20 de l'article)"


Modele : C'est le GRU-SVM, décrit en section 2.4.1 du PDF comme une architecture combinant GRU (un type de RNN) et SVM pour la classification binaire.

Liste des variables impliquées : Ce sont les 30 features du dataset WDBC (section 2.2). Elles sont calculées à partir d'images FNA (fine needle aspirate) d'une masse mammaire. Chaque feature a 3 variantes : mean (moyenne), se (standard error), worst (les 3 plus grandes valeurs). Pas de variables supplémentaires spécifiques au modèle ; il utilise tout le dataset standardisé (via Eq. 1 pour éviter les biais).

Listes des paramètres et hyperparamètres : Extraits du Table 1 (page 3). L'article dit que tous les hyperparamètres ont été assignés manuellement. Le "Norm: L2" fait référence à la variante L2-SVM (Eq. 20), et Adam est l'optimizer pour minimiser la loss (pas SGD comme pour d'autres modèles). Pas plus de détails dans l'article sur d'autres params (ex. : weights initialisés arbitrairement).